<a href="https://colab.research.google.com/github/YaoGroup/DIFFICE.jax/blob/main/tutorial/pinn_syndata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
hello('world')